# 0.3 IBKR Connection Test

Test IB Gateway connection and option quote fetching for Phase 0 validation.

**Prerequisites:**
- IB Gateway running on port 4002 (paper trading)
- API connections enabled

In [67]:
import nest_asyncio
nest_asyncio.apply()

from ib_insync import IB, Stock, Option
from datetime import datetime, timedelta
import pandas as pd

# Connection settings
IB_HOST = '127.0.0.1'
IB_PORT = 4002  # IB Gateway paper trading
CLIENT_ID = 3

## 1. Connect to IB Gateway

In [68]:
ib = IB()
ib.connect(IB_HOST, IB_PORT, clientId=CLIENT_ID)
print(f'Connected: {ib.isConnected()}')

# Use delayed market data (free, 15-min delay)
# 1 = Live, 2 = Frozen, 3 = Delayed, 4 = Delayed Frozen
ib.reqMarketDataType(1)

Connected: True


In [69]:
# Account info
print("Account Summary:")
for av in ib.accountValues():
    if av.currency == 'EUR' and av.tag in ['NetLiquidation', 'AvailableFunds', 'TotalCashValue', 'BuyingPower']:
        print(f"  {av.tag}: ${float(av.value):,.2f}")

Account Summary:
  AvailableFunds: $1,000,454.98
  BuyingPower: $6,669,699.87
  NetLiquidation: $1,000,454.98
  TotalCashValue: $1,000,000.00


## 2. Test Stock Quote

In [70]:
# Get AAPL quote
stock = Stock('AAPL', 'SMART', 'USD')
ib.qualifyContracts(stock)

ticker = ib.reqMktData(stock, '', False, False)
ib.sleep(3)

# For delayed data, use delayedBid/delayedAsk/delayedLast
spot = ticker.marketPrice()
if spot != spot or spot <= 0:
    spot = ticker.close

print(f"AAPL spot: ${spot:.2f}" if spot and spot > 0 else "AAPL spot: No data")
print(f"  Bid: {ticker.bid} ")
print(f"  Ask: {ticker.ask} ")
print(f"  Last: {ticker.last}")
print(f"  Close: {ticker.close}")

ib.cancelMktData(stock)

AAPL spot: No data
  Bid: nan 
  Ask: nan 
  Last: nan
  Close: nan


In [71]:
spot

nan

## 3. Get Option Chain Info

In [72]:
# Get available expirations and strikes
chains = ib.reqSecDefOptParams(stock.symbol, '', stock.secType, stock.conId)

# Use SMART exchange
chain = next((c for c in chains if c.exchange == 'SMART'), chains[0])

expiries = sorted(chain.expirations)[:10]
print(f"Available expiries (first 10): {expiries}")

strikes = sorted(chain.strikes)
print(f"Total strikes available: {len(strikes)}")
print(f"Strike range: {min(strikes)} - {max(strikes)}")

# Use a reasonable spot estimate if we didn't get market data
# AAPL is ~$240 as of late 2025
if not spot or spot <= 0 or spot < 100:
    spot = 240.0
    print(f"\nUsing estimated spot: ${spot}")

atm_strike = min(strikes, key=lambda s: abs(s - spot))
nearby_strikes = [s for s in strikes if abs(s - spot) / spot < 0.05]  # Within 5%
print(f"\nATM strike: {atm_strike}")
print(f"Nearby strikes: {sorted(nearby_strikes)}")

Available expiries (first 10): ['20260109', '20260116', '20260123', '20260130', '20260206', '20260213', '20260220', '20260320', '20260417', '20260515']
Total strikes available: 113
Strike range: 5.0 - 550.0

ATM strike: 5.0
Nearby strikes: []


## 4. Fetch ATM Straddle Quote

In [73]:
# Get nearest expiry straddle
expiry = expiries[0]

call = Option('AAPL', expiry, atm_strike, 'C', 'SMART')
put = Option('AAPL', expiry, atm_strike, 'P', 'SMART')
ib.qualifyContracts(call, put)
ib.reqMarketDataType(1)

call_ticker = ib.reqMktData(call, '', False, False)
put_ticker = ib.reqMktData(put, '', False, False)
ib.sleep(3)

print(f"ATM Straddle - {expiry}, Strike {atm_strike}")
print(f"\nCall:")
print(f"  Bid: ${call_ticker.bid:.2f}")
print(f"  Ask: ${call_ticker.ask:.2f}")
print(f"  Mid: ${(call_ticker.bid + call_ticker.ask)/2:.2f}")

print(f"\nPut:")
print(f"  Bid: ${put_ticker.bid:.2f}")
print(f"  Ask: ${put_ticker.ask:.2f}")
print(f"  Mid: ${(put_ticker.bid + put_ticker.ask)/2:.2f}")

if call_ticker.bid > 0 and put_ticker.bid > 0:
    call_mid = (call_ticker.bid + call_ticker.ask) / 2
    put_mid = (put_ticker.bid + put_ticker.ask) / 2
    straddle_mid = call_mid + put_mid
    straddle_spread = (call_ticker.ask - call_ticker.bid) + (put_ticker.ask - put_ticker.bid)
    spread_pct = straddle_spread / straddle_mid * 100
    implied_move = straddle_mid / spot * 100
    
    print(f"\nStraddle:")
    print(f"  Mid: ${straddle_mid:.2f}")
    print(f"  Spread: ${straddle_spread:.2f} ({spread_pct:.1f}%)")
    print(f"  Implied move: {implied_move:.1f}%")

ib.cancelMktData(call)
ib.cancelMktData(put)

Error 200, reqId 6: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20260109', strike=5.0, right='C', exchange='SMART')
Error 200, reqId 7: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20260109', strike=5.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20260109', strike=5.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20260109', strike=5.0, right='P', exchange='SMART')
Error 200, reqId 8: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20260109', strike=5.0, right='C', exchange='SMART')
Error 200, reqId 9: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20260109', strike=5.0, right='P', exchange='SMART')


ATM Straddle - 20260109, Strike 5.0

Call:
  Bid: $nan
  Ask: $nan
  Mid: $nan

Put:
  Bid: $nan
  Ask: $nan
  Mid: $nan


Error 300, reqId 8: Can't find EId with tickerId:8
Error 300, reqId 9: Can't find EId with tickerId:9


In [74]:
import math
from ib_insync import Option

ib.reqMarketDataType(1)
ib.sleep(0.2)

call = Option('AAPL', expiry, atm_strike, 'C', 'SMART', currency='USD')
put  = Option('AAPL', expiry, atm_strike, 'P', 'SMART', currency='USD')
ib.qualifyContracts(call, put)

call_t = ib.reqMktData(call)
put_t  = ib.reqMktData(put)
ib.sleep(2)

def pick_bid_ask(t):
    bid = t.bid
    ask = t.ask
    if bid is None or (isinstance(bid, float) and math.isnan(bid)):
        bid = getattr(t, 'delayedBid', float('nan'))
    if ask is None or (isinstance(ask, float) and math.isnan(ask)):
        ask = getattr(t, 'delayedAsk', float('nan'))
    return bid, ask

cbid, cask = pick_bid_ask(call_t)
pbid, pask = pick_bid_ask(put_t)

print("call bid/ask:", cbid, cask, "mktDataType:", getattr(call_t, 'marketDataType', None))
print("put  bid/ask:", pbid, pask, "mktDataType:", getattr(put_t, 'marketDataType', None))

Error 200, reqId 10: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20260109', strike=5.0, right='C', exchange='SMART', currency='USD')
Error 200, reqId 11: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20260109', strike=5.0, right='P', exchange='SMART', currency='USD')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20260109', strike=5.0, right='C', exchange='SMART', currency='USD')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20260109', strike=5.0, right='P', exchange='SMART', currency='USD')
Error 200, reqId 12: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20260109', strike=5.0, right='C', exchange='SMART', currency='USD')
Error 200, reqId 13: No security definition has been found for the request, contract: Option(symbol='AAPL'

call bid/ask: nan nan mktDataType: 1
put  bid/ask: nan nan mktDataType: 1


## 5. Fetch Greeks

In [75]:
# Request with Greeks
call_ticker = ib.reqMktData(call, '106', False, False)  # 106 = option greeks
ib.sleep(3)

greeks = call_ticker.modelGreeks
if greeks:
    print(f"Call Greeks:")
    print(f"  IV: {greeks.impliedVol*100:.1f}%")
    print(f"  Delta: {greeks.delta:.3f}")
    print(f"  Gamma: {greeks.gamma:.4f}")
    print(f"  Theta: ${greeks.theta:.3f}")
    print(f"  Vega: ${greeks.vega:.3f}")
else:
    print("Greeks not available")

ib.cancelMktData(call)

Error 200, reqId 14: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20260109', strike=5.0, right='C', exchange='SMART', currency='USD')


Greeks not available


In [76]:
def get_straddle_quote(symbol: str, expiry: str = None, fallback_spot: float = None):
    """Get ATM straddle quote for a symbol."""
    stock = Stock(symbol, 'SMART', 'USD')
    ib.qualifyContracts(stock)
    
    # Get spot - try delayed fields
    ticker = ib.reqMktData(stock, '', False, False)
    ib.sleep(3)
    spot = ticker.marketPrice()
    if spot != spot:  # nan
        spot = ticker.delayedLast
    if spot != spot or spot <= 0:
        spot = ticker.close
    ib.cancelMktData(stock)
    
    if (not spot or spot <= 0 or spot != spot) and fallback_spot:
        spot = fallback_spot
        print(f"{symbol}: Using fallback spot ${spot}")
    elif not spot or spot <= 0 or spot != spot:
        print(f"{symbol}: Could not get spot price")
        return None
    
    # Get chain
    chains = ib.reqSecDefOptParams(stock.symbol, '', stock.secType, stock.conId)
    if not chains:
        print(f"{symbol}: No option chain")
        return None
    
    chain = next((c for c in chains if c.exchange == 'SMART'), chains[0])
    
    # Use provided expiry or nearest
    if expiry is None:
        expiry = sorted(chain.expirations)[0]
    elif expiry not in chain.expirations:
        print(f"{symbol}: Expiry {expiry} not available")
        return None
    
    # ATM strike
    strikes = sorted(chain.strikes)
    atm_strike = min(strikes, key=lambda s: abs(s - spot))
    
    # Get options - specify tradingClass to avoid ambiguity
    call = Option(symbol, expiry, atm_strike, 'C', 'SMART', tradingClass=symbol)
    put = Option(symbol, expiry, atm_strike, 'P', 'SMART', tradingClass=symbol)
    qualified = ib.qualifyContracts(call, put)
    
    if len(qualified) < 2:
        print(f"{symbol}: Could not qualify options (got {len(qualified)})")
        return None
    
    call_ticker = ib.reqMktData(call, '', False, False)
    put_ticker = ib.reqMktData(put, '', False, False)
    ib.sleep(3)
    
    # Use delayed bid/ask if regular not available
    def get_bid(t):
        if t.bid and t.bid == t.bid and t.bid > 0:
            return t.bid
        if t.delayedBid and t.delayedBid == t.delayedBid and t.delayedBid > 0:
            return t.delayedBid
        return None
    
    def get_ask(t):
        if t.ask and t.ask == t.ask and t.ask > 0:
            return t.ask
        if t.delayedAsk and t.delayedAsk == t.delayedAsk and t.delayedAsk > 0:
            return t.delayedAsk
        return None
    
    result = {
        'symbol': symbol,
        'spot': spot,
        'expiry': expiry,
        'strike': atm_strike,
        'call_bid': get_bid(call_ticker),
        'call_ask': get_ask(call_ticker),
        'put_bid': get_bid(put_ticker),
        'put_ask': get_ask(put_ticker),
    }
    
    cb = result['call_bid'] or 0
    ca = result['call_ask'] or 0
    pb = result['put_bid'] or 0
    pa = result['put_ask'] or 0
    
    if cb > 0 and ca > 0 and pb > 0 and pa > 0:
        call_mid = (cb + ca) / 2
        put_mid = (pb + pa) / 2
        result['straddle_mid'] = call_mid + put_mid
        result['straddle_spread'] = (ca - cb) + (pa - pb)
        result['spread_pct'] = result['straddle_spread'] / result['straddle_mid'] * 100
        result['implied_move'] = result['straddle_mid'] / spot * 100
    
    ib.cancelMktData(call)
    ib.cancelMktData(put)
    
    return result

In [77]:
def get_straddle_quote(symbol: str, expiry: str = None, fallback_spot: float = None):
    """Get ATM straddle quote for a symbol."""
    stock = Stock(symbol, 'SMART', 'USD')
    ib.qualifyContracts(stock)
    
    # Get spot
    ticker = ib.reqMktData(stock, '', False, False)
    ib.sleep(3)
    spot = ticker.marketPrice()
    if spot != spot:  # nan check
        spot = ticker.last
    if spot != spot or spot <= 0:
        spot = ticker.close
    ib.cancelMktData(stock)
    
    if (not spot or spot <= 0) and fallback_spot:
        spot = fallback_spot
        print(f"{symbol}: Using fallback spot ${spot}")
    elif not spot or spot <= 0:
        print(f"{symbol}: Could not get spot price")
        return None
    
    # Get chain
    chains = ib.reqSecDefOptParams(stock.symbol, '', stock.secType, stock.conId)
    if not chains:
        print(f"{symbol}: No option chain")
        return None
    
    chain = next((c for c in chains if c.exchange == 'SMART'), chains[0])
    
    # Use provided expiry or nearest
    if expiry is None:
        expiry = sorted(chain.expirations)[0]
    elif expiry not in chain.expirations:
        print(f"{symbol}: Expiry {expiry} not available")
        return None
    
    # ATM strike
    strikes = sorted(chain.strikes)
    atm_strike = min(strikes, key=lambda s: abs(s - spot))
    
    # Get options - specify tradingClass to avoid ambiguity
    call = Option(symbol, expiry, atm_strike, 'C', 'SMART', tradingClass=symbol)
    put = Option(symbol, expiry, atm_strike, 'P', 'SMART', tradingClass=symbol)
    qualified = ib.qualifyContracts(call, put)
    
    if len(qualified) < 2:
        print(f"{symbol}: Could not qualify options (got {len(qualified)})")
        return None
    
    call_ticker = ib.reqMktData(call, '', False, False)
    put_ticker = ib.reqMktData(put, '', False, False)
    ib.sleep(3)
    
    result = {
        'symbol': symbol,
        'spot': spot,
        'expiry': expiry,
        'strike': atm_strike,
        'call_bid': call_ticker.bid if call_ticker.bid == call_ticker.bid else None,
        'call_ask': call_ticker.ask if call_ticker.ask == call_ticker.ask else None,
        'put_bid': put_ticker.bid if put_ticker.bid == put_ticker.bid else None,
        'put_ask': put_ticker.ask if put_ticker.ask == put_ticker.ask else None,
    }
    
    cb = result['call_bid'] or 0
    ca = result['call_ask'] or 0
    pb = result['put_bid'] or 0
    pa = result['put_ask'] or 0
    
    if cb > 0 and ca > 0 and pb > 0 and pa > 0:
        call_mid = (cb + ca) / 2
        put_mid = (pb + pa) / 2
        result['straddle_mid'] = call_mid + put_mid
        result['straddle_spread'] = (ca - cb) + (pa - pb)
        result['spread_pct'] = result['straddle_spread'] / result['straddle_mid'] * 100
        result['implied_move'] = result['straddle_mid'] / spot * 100
    
    ib.cancelMktData(call)
    ib.cancelMktData(put)
    
    return result

Error 300, reqId 14: Can't find EId with tickerId:14


In [78]:
# Test with a few symbols - include fallback prices for delayed data issues
test_symbols = [
    ('AAPL', 240),
    ('TSLA', 410),
    ('NVDA', 140),
    ('META', 600),
    ('GOOGL', 195),
]

results = []
for sym, fallback in test_symbols:
    print(f"Fetching {sym}...")
    result = get_straddle_quote(sym, fallback_spot=fallback)
    if result:
        results.append(result)

if results:
    df = pd.DataFrame(results)
    print("\nStraddle Quotes:")
    display_cols = ['symbol', 'spot', 'expiry', 'strike', 'straddle_mid', 'spread_pct', 'implied_move']
    print(df[[c for c in display_cols if c in df.columns]].round(2).to_string(index=False))
else:
    print("\nNo results - may need live market data subscription or market hours")

Fetching AAPL...


Error 200, reqId 18: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20260109', strike=5.0, right='C', exchange='SMART', tradingClass='AAPL')
Error 200, reqId 19: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20260109', strike=5.0, right='P', exchange='SMART', tradingClass='AAPL')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20260109', strike=5.0, right='C', exchange='SMART', tradingClass='AAPL')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20260109', strike=5.0, right='P', exchange='SMART', tradingClass='AAPL')


AAPL: Could not qualify options (got 0)
Fetching TSLA...


Error 200, reqId 23: No security definition has been found for the request, contract: Option(symbol='TSLA', lastTradeDateOrContractMonth='20260109', strike=5.0, right='C', exchange='SMART', tradingClass='TSLA')
Error 200, reqId 24: No security definition has been found for the request, contract: Option(symbol='TSLA', lastTradeDateOrContractMonth='20260109', strike=5.0, right='P', exchange='SMART', tradingClass='TSLA')
Unknown contract: Option(symbol='TSLA', lastTradeDateOrContractMonth='20260109', strike=5.0, right='C', exchange='SMART', tradingClass='TSLA')
Unknown contract: Option(symbol='TSLA', lastTradeDateOrContractMonth='20260109', strike=5.0, right='P', exchange='SMART', tradingClass='TSLA')


TSLA: Could not qualify options (got 0)
Fetching NVDA...


Error 200, reqId 28: No security definition has been found for the request, contract: Option(symbol='NVDA', lastTradeDateOrContractMonth='20260109', strike=0.5, right='C', exchange='SMART', tradingClass='NVDA')
Error 200, reqId 29: No security definition has been found for the request, contract: Option(symbol='NVDA', lastTradeDateOrContractMonth='20260109', strike=0.5, right='P', exchange='SMART', tradingClass='NVDA')
Unknown contract: Option(symbol='NVDA', lastTradeDateOrContractMonth='20260109', strike=0.5, right='C', exchange='SMART', tradingClass='NVDA')
Unknown contract: Option(symbol='NVDA', lastTradeDateOrContractMonth='20260109', strike=0.5, right='P', exchange='SMART', tradingClass='NVDA')


NVDA: Could not qualify options (got 0)
Fetching META...
Fetching GOOGL...

Straddle Quotes:
symbol  spot   expiry  strike
  META   NaN 20260116   600.0
 GOOGL   NaN 20260116   300.0


## 7. Test Order Placement (Paper Trading)

Place a small test order to validate execution flow.

In [ ]:
#WARNING: This will place a real order in paper trading
#Uncomment to test

# from ib_insync import LimitOrder

# # Use a low limit price that won't fill immediately
# test_symbol = 'AAPL'
# test_expiry = expiries[0]
# test_strike = atm_strike

# call = Option(test_symbol, test_expiry, test_strike, 'C', 'SMART')
# ib.qualifyContracts(call)

# # Place limit order well below market
# order = LimitOrder('BUY', 1, 0.01)  # $0.01 limit - won't fill
# trade = ib.placeOrder(call, order)

# print(f"Order placed: {trade.order.orderId}")
# print(f"Status: {trade.orderStatus.status}")

# # Cancel it
# ib.cancelOrder(trade.order)
# print("Order cancelled")

## 8. Cleanup

In [79]:
ib.disconnect()
print("Disconnected from IB Gateway")

Disconnected from IB Gateway


## Summary

If all cells ran successfully:
- IB Gateway connection works
- Can fetch stock quotes
- Can fetch option chains and quotes
- Can compute straddle pricing and implied moves
- Ready for Phase 0 execution validation

**Next steps:**
1. Screen upcoming earnings for tradeable candidates
2. Place small test orders to validate fill model
3. Log all execution details for Phase 0 analysis